# Content Based Recommendation

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
from IPython.display import Markdown, display
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

## Data Preprocessing

In [3]:
DATA_DIR = os.path.abspath('./datasets/20_news')
OUTPUT_DIR = os.path.join('./datasets/output')

In [4]:
RAW_DATA_DIR = DATA_DIR
OUTPUT_DIR = os.path.join('./datasets/output')

In [5]:
classes = os.listdir(RAW_DATA_DIR)
columns=['id','class','text']

data_dict = {
    'id':[],
    'class':[],
    'text': []
}


In [6]:
for label in classes:
    files = os.path.join(RAW_DATA_DIR,label)
    for f in tqdm(os.listdir(files)):
        id_ = f.split('.')[0]
        try:
            with open(os.path.join(files,f),encoding='utf-8') as content:
                text = content.read().strip()
            #Adding corresponding data to the dictionary
            data_dict['id'].append(id_)
            data_dict['class'].append(label)
            data_dict['text'].append(text)
        except:
            print(label,id_,"Failed to load")


 31%|███       | 102/330 [00:00<00:00, 1013.42it/s]

Interview 310 Failed to load
Interview 257 Failed to load
Interview 293 Failed to load
Interview 289 Failed to load
Interview 244 Failed to load
Interview 275 Failed to load
Interview 242 Failed to load
Interview 260 Failed to load
Interview 311 Failed to load
Interview 309 Failed to load
Interview 109 Failed to load
Interview 318 Failed to load
Interview 328 Failed to load
Interview 256 Failed to load
Interview 8 Failed to load
Interview 236 Failed to load
Interview 285 Failed to load
Interview 122 Failed to load
Interview 255 Failed to load
Interview 300 Failed to load
Interview 286 Failed to load
Interview 249 Failed to load
Interview 329 Failed to load
Interview 261 Failed to load
Interview 313 Failed to load
Interview 297 Failed to load
Interview 291 Failed to load
Interview 273 Failed to load
Interview 298 Failed to load
Interview 245 Failed to load
Interview 321 Failed to load
Interview 296 Failed to load
Interview 253 Failed to load
Interview 254 Failed to load
Interview 240 Fa

  0%|          | 0/550 [00:00<?, ?it/s]

Interview 269 Failed to load
Interview 307 Failed to load
Interview 282 Failed to load
Interview 241 Failed to load
Interview 259 Failed to load
Interview 1 Failed to load
Interview 268 Failed to load
Interview 247 Failed to load
Interview 239 Failed to load
Interview 302 Failed to load
Interview 314 Failed to load
Interview 263 Failed to load
Interview 305 Failed to load
Interview 235 Failed to load
Interview 326 Failed to load
Interview 317 Failed to load
Interview 327 Failed to load
Interview 270 Failed to load
Interview 292 Failed to load
Interview 281 Failed to load
Interview 252 Failed to load
Interview 322 Failed to load
Interview 308 Failed to load
Interview 287 Failed to load
Interview 2 Failed to load
Interview 4 Failed to load
Interview 280 Failed to load
Interview 316 Failed to load
Interview 237 Failed to load
Interview 301 Failed to load
Interview 243 Failed to load
Interview 262 Failed to load
Interview 238 Failed to load
Interview 290 Failed to load
Interview 267 Failed

  0%|          | 0/500 [00:00<?, ?it/s]

...

World 247 Failed to load
World 239 Failed to load
World 218 Failed to load
World 302 Failed to load
World 263 Failed to load
World 305 Failed to load
World 78 Failed to load
World 235 Failed to load
World 270 Failed to load
World 222 Failed to load
World 292 Failed to load
World 281 Failed to load
World 252 Failed to load
World 308 Failed to load
World 287 Failed to load
World 2 Failed to load
World 4 Failed to load
World 280 Failed to load
World 237 Failed to load
World 301 Failed to load
World 243 Failed to load
World 262 Failed to load
World 238 Failed to load
World 290 Failed to load
World 267 Failed to load
World 3 Failed to load
World 228 Failed to load
World 295 Failed to load
World 284 Failed to load
World 221 Failed to load
World 248 Failed to load
World 250 Failed to load
World 278 Failed to load
World 231 Failed to load
World 265 Failed to load
World 294 Failed to load
World 258 Failed to load
Employment 257 Failed to load
Employment 293 Failed to load
Employment 289 Failed 

  0%|          | 0/634 [00:00<?, ?it/s]

Employment 211 Failed to load
Employment 232 Failed to load
Employment 225 Failed to load
Employment 303 Failed to load
Employment 274 Failed to load
Employment 212 Failed to load
Employment 227 Failed to load
Employment 269 Failed to load
Employment 241 Failed to load
Employment 259 Failed to load
Employment 268 Failed to load
Employment 103 Failed to load
Employment 247 Failed to load
Employment 239 Failed to load
Employment 206 Failed to load
Employment 218 Failed to load
Employment 302 Failed to load
Employment 263 Failed to load
Employment 235 Failed to load
Employment 214 Failed to load
Employment 270 Failed to load
Employment 222 Failed to load
Employment 292 Failed to load
Employment 208 Failed to load
Employment 281 Failed to load
Employment 252 Failed to load
Employment 210 Failed to load
Employment 287 Failed to load
Employment 280 Failed to load
Employment 216 Failed to load
Employment 237 Failed to load
Employment 301 Failed to load
Employment 243 Failed to load
Employment

100%|██████████| 634/634 [00:00<00:00, 2457.81it/s]

Entertainment 584 Failed to load
Entertainment 547 Failed to load
Entertainment 583 Failed to load
Entertainment 537 Failed to load
Entertainment 568 Failed to load
Entertainment 546 Failed to load
Entertainment 563 Failed to load
Entertainment 524 Failed to load
Entertainment 572 Failed to load
Entertainment 514 Failed to load
Entertainment 508 Failed to load
Entertainment 577 Failed to load
Entertainment 529 Failed to load
Entertainment 543 Failed to load
Entertainment 566 Failed to load
Entertainment 505 Failed to load
Entertainment 506 Failed to load
Entertainment 556 Failed to load
Entertainment 557 Failed to load
Entertainment 553 Failed to load
Entertainment 544 Failed to load
Entertainment 573 Failed to load
Entertainment 511 Failed to load
Entertainment 528 Failed to load
Entertainment 551 Failed to load
Entertainment 364 Failed to load
Entertainment 575 Failed to load
Entertainment 523 Failed to load
Entertainment 565 Failed to load
Entertainment 581 Failed to load
Entertainm

In [20]:
df = pd.DataFrame(data_dict,columns=columns)
df.columns
df

,id,class,text
0,182,Interview,"﻿विडम्बना, ओलीको सत्ता सवारी पनि सिंहदरबारमा फ..."
1,127,Interview,﻿नेपाली कांग्रेसको सभापतिमा रामचन्द्र पौडेलको ...
2,85,Interview,﻿\nदुर्भाग्य नै भन्नुपर्छ हाम्रो प्राथमिकता पे...
3,162,Interview,"﻿नयनराज पाण्डे, लेखक\nविमोचन हुनै लागेको तपाईं..."
4,13,Interview,"﻿- भवन भट्ट, उपाध्यक्ष, गैर आवासीय नेपाली संघ\..."
...,...,...,...
5471,68,Entertainment,﻿उनले ‘टिमवर्क’ र ‘लिडरसिप’ फिल्ममेकिङको महत्व...
5472,605,Entertainment,रुपा बनिन् हेरिटेज क्वीन नेपाल\n\n\nराजधानीको ...
5473,294,Entertainment,﻿प्रकाश र किरणको अर्थ एउटै भए पनि कालो पोथी का...
5474,609,Entertainment,दानापुर तालमा पर्यटक लोभिँदै\n\nअहिले पर्यटक ल...


In [8]:
df.dropna(inplace=True) # drop rows with invalid values

## Processing data

In [9]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [22]:
nepali_stop_words = set(stopwords.words('nepali'))

tf = TfidfVectorizer(stop_words=nepali_stop_words,encoding='utf-8',decode_error='ignore')
tf = TfidfVectorizer(tokenizer= lambda x: x.split(" "),
                                  sublinear_tf=True, encoding='utf-8',
                                  decode_error='ignore',
                                  max_df=0.5,
                                  min_df=10,
                                  stop_words=nepali_stop_words)

tfidf_matrix = tf.fit_transform(df['text'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

In [23]:
for idx, row in df.iterrows():
    idx = int(idx)
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], df['id'][i], df['class'][i]) for i in similar_indices]

    results[row['id']] = similar_items[1:]
    
print('Completed!')
similar_indices = cosine_similarities[idx].argsort()[:-100:-1]

Completed!


## Displaying Results

In [12]:
def printBold(string):
    display(Markdown(string))

def item(id):
    id = str(id)
    return {'text': df.loc[df['id'] == id]['text'].tolist()[0].split(' - ')[0],
           'class': df.loc[df['id'] == id]['class'].tolist()[0].split(' - ')[0]
           }

def recommender(item_id, num): 
    print("\n")
    printBold("**Recommendation System**")
    print("The given news " + "'"+ item(item_id)['text'][:100] + "..." + "'" + " is of Category: " + str(item(item_id)['class'][:100]))
    
    print("\nThe " + str(num) + " most similar news in ascending order is given below: ")
    
    recs = results[str(item_id)][:num]
    for index, rec in enumerate(recs):
        print("\n")
        print(str(index+1) + ") " + "Category: " + str(rec[2]) + "\n\n" + "News: " + "'" + item(rec[1])['text'][:200] + "..." + "'" +" (score:" + str(rec[0]) + ")")

# Extract the news (that the user is currently interacting with) and provide 8 recommended news
recommender(item_id=120, num=10)

**Recommendation System**

The given news '﻿
वसन्त अधिकारी निर्देशक/प्रवक्ता, यातायात व्यवस्था विभाग
सार्वजनिक यातायात सुधार्न के गर्नुपर्छ ?
न...' is of Category: Interview

The 10 most similar news in ascending order is given below: 


1) Category: Sports

News: '﻿बिरेन्द्र बजार, ३० जेठ । नेपाल आदिवासी जनजाती महासंघ सप्तरीका अध्यक्ष बृजमान तामाङले आगामी असार १४ गते हुने स्थानिय तहको चुनावमा धेरै भोजभतेर गर्ने उम्मेदवारलाई भोट नदिन आग्रह गरेका छन् ।
नेपाल आदिवा...' (score:0.2272281713105735)


2) Category: Sports

News: '﻿भारतीय क्रिकेट टिमका कुनै एक समय थियो जब कपिल देव र सुनील गावस्कर जस्ता दिग्गज कप्तानले विरोधी टिम र आफ्नो खेलमा मात्र ध्यान दिन्थे। त्यसपछि सौरव गांगुली जस्ता कप्तान आए जसले भारतीय खेलाडीलाई विरोधी ...' (score:0.1275688655313746)


3) Category: Sports

News: '﻿राजविराज, १५ जेठ । राजविराज नगरपालिकामा सञ्चालित सहरी बाल शिक्षा कार्यक्रमको एक टोलीद्वारा सोमवार अनुगमन गरिएको छ ।
श्रमिक वा विद्यालय नगएका बालबालिकालाई विद्यालयमा भर्ना गराउने उद्देश्यले युनिसेफको ...' (score:0.12727302269389637)


4)

In [18]:
import pickle
with open('similarity_matrix.pkl','wb') as f:
    pickle.dump(results,f)